In [1]:
import tensorflow as tf

In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv('train.csv')

In [4]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
dataset=dataset.dropna()

In [10]:
x=dataset.drop('label',axis=1)

In [11]:
y=dataset['label']

In [12]:
from tensorflow.keras.layers import Embedding

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
from tensorflow.keras.models import Sequential

In [15]:
from tensorflow.keras.preprocessing.text import one_hot

In [16]:
from tensorflow.keras.layers import LSTM

In [17]:
from tensorflow.keras.layers import Dense

In [18]:
vocabulary=5000

In [19]:
messages=x.copy()

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\itism\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [24]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
onehot=[one_hot(words,vocabulary)for words in corpus] 
onehot

[[4327, 2213, 1981, 588, 3360, 3777, 1886, 3241, 329, 1163],
 [674, 3259, 580, 631, 45, 2871, 2933],
 [615, 2317, 1561, 2123],
 [53, 584, 3488, 4373, 3379, 4253],
 [1769, 45, 1475, 4136, 59, 4143, 45, 2746, 4892, 4637],
 [3736,
  75,
  2718,
  2591,
  181,
  4609,
  160,
  1265,
  2586,
  3668,
  1963,
  572,
  1285,
  638,
  2933],
 [2490, 2043, 497, 4758, 1822, 4233, 2414, 535, 3857, 4863, 2390],
 [1710, 4167, 4019, 4395, 4047, 4190, 4609, 416, 3857, 4863, 2390],
 [3569, 1958, 1314, 1792, 972, 1629, 462, 4987, 4609, 4324],
 [4648, 4939, 2732, 1258, 3758, 2521, 53, 1744],
 [4979, 619, 3986, 3792, 4534, 3666, 4512, 4262, 1287, 2036, 797],
 [4373, 4107, 3360, 1629, 4609, 4047],
 [2597, 2168, 3026, 2303, 811, 4543, 181, 3501, 2466],
 [1886, 2396, 4429, 2511, 4306, 2341, 3490, 3857, 4863, 2390],
 [2123, 1631, 4629, 1058, 4517, 3857, 4863, 2390],
 [129, 1982, 166, 161, 2725, 1764, 1613, 580, 2066, 2230],
 [1271, 1260, 3259],
 [4254, 2374, 3428, 601, 4609, 3739, 696, 2933],
 [2768, 2342, 58

In [29]:
length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=length)
print(embedded_docs)

[[   0    0    0 ... 3241  329 1163]
 [   0    0    0 ...   45 2871 2933]
 [   0    0    0 ... 2317 1561 2123]
 ...
 [   0    0    0 ... 3857 4863 2390]
 [   0    0    0 ... 3560 3879 3292]
 [   0    0    0 ... 4306 1457 1816]]


In [54]:
features=40
model=Sequential()
model.add(Embedding(vocabulary,features,input_length=length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [57]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
192/192 [==============================] - 5s 28ms/step - loss: 0.3290 - accuracy: 0.8500 - val_loss: 0.2037 - val_accuracy: 0.9181
Epoch 2/15
192/192 [==============================] - 4s 23ms/step - loss: 0.1373 - accuracy: 0.9453 - val_loss: 0.1944 - val_accuracy: 0.9239
Epoch 3/15
192/192 [==============================] - 4s 23ms/step - loss: 0.0934 - accuracy: 0.9631 - val_loss: 0.2068 - val_accuracy: 0.9167
Epoch 4/15
192/192 [==============================] - 5s 25ms/step - loss: 0.0643 - accuracy: 0.9776 - val_loss: 0.2297 - val_accuracy: 0.9185
Epoch 5/15
192/192 [==============================] - 4s 20ms/step - loss: 0.0377 - accuracy: 0.9865 - val_loss: 0.3062 - val_accuracy: 0.9188
Epoch 6/15
192/192 [==============================] - 4s 22ms/step - loss: 0.0239 - accuracy: 0.9922 - val_loss: 0.3506 - val_accuracy: 0.9153
Epoch 7/15
192/192 [==============================] - 5s 28ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.3533 - val_accuracy: 0.9145

In [49]:
y_pred=model.predict_classes(X_test)

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred)

array([[3087,  332],
       [ 180, 2436]], dtype=int64)

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9151615575807788